In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow import keras
from tensorflow.keras.optimizers import SGD
import warnings
warnings.filterwarnings("ignore")

In [19]:
learning_rate = 0.0001
buffer_size = 519
batch_size = 16
epochs = 500
img_size = 224
key = 18

In [3]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

training_set = train_datagen.flow_from_directory('output/train/',
                                                 target_size=(224,224),
                                                 batch_size=16)

test_set = test_datagen.flow_from_directory('output/val',
                                            target_size=(224,224),
                                            batch_size=16)

Found 1885 images belonging to 18 classes.
Found 481 images belonging to 18 classes.


In [4]:
def InceptionResNetV2_model():

    engine = tf.keras.applications.Xception(
        # Freezing the weights of the top layer in the InceptionResNetV2 pre-traiined model
        include_top = False,

        # Use Imagenet weights
        weights = 'imagenet',

        # Define input shape to 224x224x3
        input_shape = (img_size , img_size , 3),

    )

    x = tf.keras.layers.GlobalAveragePooling2D(name = 'avg_pool')(engine.output)
    x =Dropout(0.85)(x)
    out = tf.keras.layers.Dense(key, activation = 'softmax', name = 'dense_output')(x)


    # Build the Keras model

    model = tf.keras.models.Model(inputs = engine.input, outputs = out)
    # Compile the model

    model.compile(
        # Set optimizer to Adam(0.0001)
        optimizer = tf.keras.optimizers.Adam(learning_rate= 3e-4),
        #optimizer= SGD(lr=3e-4, momentum=0.9),
        # Set loss to binary crossentropy
        #loss = tf.keras.losses.sparse_categorical_crossentropy,
        loss = tf.keras.losses.CategoricalCrossentropy(),
        # Set metrics to accuracy
        metrics = ['accuracy']
    )

    return model

In [5]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                                                            patience=2,
                                                            verbose=2,
                                                            factor=0.5,
                                                            min_lr=0.00001)
reduce_lr =  keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, min_lr=0.00001)

In [6]:
def train():
    time_start = time.time()
    
    model = InceptionResNetV2_model()
    
    model.summary()
    history = model.fit(training_set, epochs= epochs , verbose = 2, validation_data = test_set,
                       callbacks=[early_stopping, reduce_lr , learning_rate_reduction])
    
    

    model.save_weights('layer22-wskindiseases1.h5')
    model.save('layer22-skindiseases1.h5')
    
    print('Model saved.')
    
    time_end = time.time()
    print('Training Time:', time_end - time_start)
    print('\n')

    return history


In [7]:
def test():
    #test_labels = np.array(test_labels)

    from tensorflow import keras
    print('Testing:')
    mod = InceptionResNetV2_model()
    mod.load_weights('Layer22-wskindiseases1.h5')
    mod.evaluate(test_set)
    
    #prob = mod.predict(test_set)
    #predIdxs = np.argmax(prob, axis=1) 


    print('\n')
    #print(classification_report(test_set.labels, predIdxs,target_names = key, digits=5))
    return predIdxs, prob, mod 

In [8]:
if __name__ == "__main__":

    train_history = train() 
    predIdxs,prob,  model = test()
    


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

Epoch 1/500
118/118 - 64s - loss: 2.0671 - accuracy: 0.3798 - val_loss: 1.5083 - val_accuracy: 0.5759
Epoch 2/500
118/118 - 56s - loss: 1.1510 - accuracy: 0.6573 - val_loss: 0.9885 - val_accuracy: 0.6965
Epoch 3/500
118/118 - 56s - loss: 0.7636 - accuracy: 0.7655 - val_loss: 1.0398 - val_accuracy: 0.7422
Epoch 4/500
118/118 - 56s - loss: 0.5305 - accuracy: 0.8329 - val_loss: 0.8439 - val_accuracy: 0.8025
Epoch 5/500
118/118 - 56s - loss: 0.4258 - accuracy: 0.8727 - val_loss: 0.9264 - val_accuracy: 0.7692
Epoch 6/500
118/118 - 56s - loss: 0.3542 - accuracy: 0.8875 - val_loss: 0.7944 - val_accuracy: 0.7859

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0001500000071246177.
Epoch 7/500
118/118 - 56s - loss: 0.2270 - accuracy: 0.9347 - val_loss: 0.6517 - val_accuracy: 0.8316
Epoch 8/500
118/118 - 57s - loss: 0.1518 - accuracy: 0.9533 - val_loss: 0.5540 - val_accuracy: 0.8607
Epoch 9/500
118/118 - 56s - loss: 0.1141 - accuracy: 0.9719 - val_loss: 0.5856 - val_accuracy: 0.8669
E

C:\Users\laptop\AppData\Roaming\Python\Python38\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Model saved.
Training Time: 1196.1315298080444


Testing:
31/31 [==============================] - 4s 94ms/step - loss: 0.5876 - accuracy: 0.8877




NameError: name 'predIdxs' is not defined

In [20]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
input_path = 'Acne and Rosacea/'
train_data = image_dataset_from_directory(directory=input_path,
                                          batch_size=16,
                                          image_size=(224, 224))
key = train_data.class_names

Found 2366 files belonging to 18 classes.


In [21]:
def read_directory():
    data_filenames = []
    data_labels = []
    for i in range (len(key)):
        for filename in os.listdir(input_path + key[i]):
                data_filenames.append(input_path +key[i]+'/' + filename)
                data_labels.append(i)
  
    return  data_filenames, data_labels

In [22]:
def build_test_tfrecord(train_filenames, train_labels):  # Generate TFRecord of training set 
    with tf.io.TFRecordWriter(test_tfrecord)as writer:
        for filename, label in zip(train_filenames, train_labels):
            image = open(filename, 'rb').read()

            feature = {
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),  # img > Bytes
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))  # label > Int
            }

            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())

In [23]:
def _parse_example(example_string):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }

    feature_dict = tf.io.parse_single_example(example_string, feature_description)
    feature_dict['image'] = tf.io.decode_png(feature_dict['image'], channels=3)
    feature_dict['image'] = tf.image.resize(feature_dict['image'], [img_size, img_size]) / 255.0
    return feature_dict['image'], feature_dict['label']

In [24]:
def get_test_dataset(test_tfrecord):
    raw_test_dataset = tf.data.TFRecordDataset(test_tfrecord)
    test_dataset = raw_test_dataset.map(_parse_example)

    return test_dataset

In [25]:
def data_Preprocessing( test_dataset):
    test_dataset = test_dataset.batch(batch_size)
    test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return  test_dataset

In [26]:
def InceptionResNetV2_model():

    engine = tf.keras.applications.Xception(
        # Freezing the weights of the top layer in the InceptionResNetV2 pre-traiined model
        include_top = False,

        # Use Imagenet weights
        weights = 'imagenet',

        # Define input shape to 224x224x3
        input_shape = (img_size , img_size , 3),

    )

    x = tf.keras.layers.GlobalAveragePooling2D(name = 'avg_pool')(engine.output)
    x =Dropout(0.85)(x)
    out = tf.keras.layers.Dense(len(key), activation = 'softmax', name = 'dense_output')(x)


    # Build the Keras model

    model = tf.keras.models.Model(inputs = engine.input, outputs = out)
    # Compile the model

    model.compile(
        # Set optimizer to Adam(0.0001)
        optimizer = tf.keras.optimizers.Adam(learning_rate= 3e-4),
        #optimizer= SGD(lr=0.0001, momentum=0.9),
        # Set loss to binary crossentropy
        loss = tf.keras.losses.sparse_categorical_crossentropy,

        # Set metrics to accuracy
        metrics = ['accuracy']
    )


    return model

In [27]:
def test(test_labels):
    test_labels = np.array(test_labels)

    from tensorflow import keras
    print('Testing:')
    mod = InceptionResNetV2_model()
    mod.load_weights('layer22-wskindiseases1.h5')
    mod.evaluate(test_dataset)
    
    prob = mod.predict(test_dataset)
    predIdxs = np.argmax(prob, axis=1) 


    print('\n')
    print(classification_report(test_labels, predIdxs,target_names = key, digits=5))
    return test_labels, predIdxs, prob, mod

In [28]:
if __name__ == "__main__":
    
    test_filenames, test_labels = read_directory()
    test_tfrecord = 'XRay_test.tfrecords'
    build_test_tfrecord(test_filenames, test_labels)


    test_dataset = get_test_dataset(test_tfrecord)



    test_dataset = data_Preprocessing(test_dataset) 

    test_labels, predIdxs,prob, Model = test(test_labels)
    

Testing:
149/149 [==============================] - 20s 103ms/step - loss: 0.1219 - accuracy: 0.9748


                          precision    recall  f1-score   support

                    Acne    0.97426   0.98148   0.97786       270
          Acne Infantile    0.96471   0.97619   0.97041        84
        Acne Open Comedo    1.00000   1.00000   1.00000        72
            Acne Steroid    0.99200   1.00000   0.99598       124
         Acne conglobata    0.93617   0.93617   0.93617        47
           Acne excoriée    1.00000   1.00000   1.00000        50
          Acne fulminans    1.00000   0.83333   0.90909        36
  Acne keloidalis nuchae    1.00000   1.00000   1.00000       118
           Acne vulgaris    0.96901   0.98286   0.97589       350
Hidradenitis suppurativa    0.97619   0.97619   0.97619       168
           Hyperhidrosis    0.97222   0.97222   0.97222        36
                   Milia    0.96373   0.96373   0.96373       193
Minocycline Pigmentation    1.00000   

In [30]:
def dermai_accuracy(prob, prob_Desicon):
    dermai = []
    for i in range(len(prob)):
        catogery =[]
        for j in range(len(prob[i])):
            if prob[i][j] > prob_Desicon:
                catogery.append(j)
        dermai.append(catogery)
    Positive = []
    for i in range(len(dermai)):
        if test_labels[i] in dermai[i]:
            Positive.append(1)
    accuracy = (len(Positive)/len(test_labels))*100  
    return  accuracy,dermai

In [31]:
accuracy,dermai = dermai_accuracy(prob, 0.10)

In [32]:
accuracy

98.23603527929441